In [3]:
!pip install langchain langchain-community rank_bm25 pypdf chromadb
!pip install unstructured['pdf'] unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.6 MB/s eta 0:00:

In [4]:
!apt-get install -y poppler-utils
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 0s (430 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123630 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
T

In [5]:
!pip install pytesseract
!pip install bitsandbytes accelerate peft safetensors sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.2 MB/s eta 0:00:00


In [6]:
!pip install pytest
!pip install -qU "langchain-chroma>=0.1.2"
!pip install -qU langchain-huggingface

In [7]:
import torch
print(torch.cuda.is_available())

True


# Split documents into chunks
The next problem is the document (each page of pdf) is probably too big to be used on its own. We need to split it into chunks

In [8]:
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
from transformers import AutoTokenizer

EMBEDDING_MODEL_NAME = 'BAAI/bge-large-en-v1.5'
print(f"Model's maximum sequence length (max tokens): {SentenceTransformer(EMBEDDING_MODEL_NAME).max_seq_length}")

def plot_docs_tokens(docs_processed, EMBEDDING_MODEL_NAME):
    tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL_NAME)
    lengths = [
        len(tokenizer.encode(doc.page_content))
        for doc in docs_processed
    ]
    plt.xlabel('Number of Tokens')
    plt.ylabel('Frequency')
    plt.title('Distribution of document lengths in count of tokens')
    plt.show()


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Model's maximum sequence length (max tokens): 512


# Advanced RAG: Hybrid_Search_RAG

In [9]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.document_loaders.pdf import PyPDFDirectoryLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.llms import HuggingFaceHub


from langchain.retrievers import BM25Retriever, EnsembleRetriever

from transformers import ( AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline, )

import os

In [9]:
!mkdir -p "pdf-data"
# !wget "https://github.com/fm6224/RAG-Hybrid-Search/raw/main/orca_paper.pdf" -P "pdf-data"

--2024-11-26 14:06:10--  https://github.com/fm6224/RAG-Hybrid-Search/raw/main/orca_paper.pdf
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/fm6224/RAG-Hybrid-Search/main/orca_paper.pdf [following]
--2024-11-26 14:06:10--  https://raw.githubusercontent.com/fm6224/RAG-Hybrid-Search/main/orca_paper.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1458242 (1.4M) [application/octet-stream]
Saving to: ‘pdf-data/orca_paper.pdf’

orca_paper.pdf      100%[===================>]   1.39M  --.-KB/s    in 0.07s   

2024-11-26 14:06:11 (19.0 MB/s) - ‘pdf-data/orca_paper.pdf’ saved [1458242/1458242]



In [10]:
!wget "https://github.com/mrdbourke/simple-local-rag/raw/main/human-nutrition-text.pdf" -P "pdf-data"

--2024-11-26 14:15:28--  https://github.com/mrdbourke/simple-local-rag/raw/main/human-nutrition-text.pdf
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mrdbourke/simple-local-rag/main/human-nutrition-text.pdf [following]
--2024-11-26 14:15:28--  https://raw.githubusercontent.com/mrdbourke/simple-local-rag/main/human-nutrition-text.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26891229 (26M) [application/octet-stream]
Saving to: ‘pdf-data/human-nutrition-text.pdf’

human-nutrition-tex 100%[===================>]  25.64M   104MB/s    in 0.2s    

2024-11-26 14:15:29 (104 MB/s) - ‘pdf-data/hu

In [11]:
# from langchain_community.document_loaders import PyPDFDirectoryLoader

# loader = PyPDFDirectoryLoader("pdf-data/")
# docs = loader.load()

In [12]:
doc_path = "/content/pdf-data/human-nutrition-text.pdf"
file = UnstructuredPDFLoader(doc_path)
docs = file.load()

In [13]:
print(len(docs)) # 51
# print(docs[0].page_content)

1


# Split documents and chunking

In [15]:
# create chunks: splitting by character
# splitter = RecursiveCharacterTextSplitter(chunk_size=800,
#                                           chunk_overlap=100)
# chunks = splitter.split_documents(docs)


# print(len(chunks)) # 226 chunks
# chunks[1].page_content

In [14]:
# create chunks: splitting by tokens
chunk_size = 512 # max token of EMBEDDING_MODEL_NAME is 512
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    AutoTokenizer.from_pretrained(EMBEDDING_MODEL_NAME),
    chunk_size=chunk_size,
    chunk_overlap=int(chunk_size/10),
)
chunks = text_splitter.split_documents(docs)

In [15]:
# chunks: splitting by tokens
# print(chunks[0])
len(chunks) # 902 chunks

902

In [16]:
import torch
print(torch.cuda.is_available())

True


In [17]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

from langchain.embeddings import HuggingFaceBgeEmbeddings
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings_model = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-large-en-v1.5",
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    query_instruction="Generate a representation for this sentence for retrieving related articles"
)
embeddings_model.query_instruction = "Generate a representation for this sentence for retrieving related articles"

# Vector store

In [18]:
# Vector store with the selected embedding model
vectorstore = Chroma.from_documents(chunks, embeddings_model)

In [19]:
vectorstore_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [20]:
keyword_retriever = BM25Retriever.from_documents(chunks)
keyword_retriever.k =  3

# Ensemble Retriever

In [33]:
ensemble_retriever = EnsembleRetriever(retrievers=[vectorstore_retriever,
                                                   keyword_retriever],
                                       weights=[0.3, 0.7])

# using HuggingfacePipeline latest version

In [22]:
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "HuggingFaceH4/zephyr-7b-beta"


# function for loading 4-bit quantized model
def load_quantized_model(model_name: str):
    """
    model_name: Name or path of the model to be loaded.
    return: Loaded quantized model.
    """
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        # load_in_4bit=True,
        torch_dtype=torch.bfloat16,
        quantization_config=bnb_config,
    )
    return model

# initializing tokenizer
def initialize_tokenizer(model_name: str):
    """
    model_name: Name or path of the model for tokenizer initialization.
    return: Initialized tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name, return_token_type_ids=False)
    tokenizer.bos_token_id = 1  # Set beginning of sentence token id
    return tokenizer

In [23]:
tokenizer = initialize_tokenizer(model_name)
model = load_quantized_model(model_name)

# specify stop token ids
stop_token_ids = [0]

# build huggingface pipeline for using zephyr-7b-beta
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    max_length=2048,
    max_new_tokens=1024,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
)

llm = HuggingFacePipeline(pipeline=pipeline)

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

# Prompt template

In [34]:
template = """
<|system|>>
You are a helpful AI Assistant that follows instructions extremely well.
Use the following context to answer user question.

Think step by step before answering the question. You will get a $100 tip if you provide correct answer.

CONTEXT: {context}
</s>
<|user|>
{query}
</s>
<|assistant|>
"""

prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()


hybrid_chain = (
    {"context": ensemble_retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)
normal_chain = (
    {"context": vectorstore_retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

## Question contains keywords

In [35]:
# page 47
print(hybrid_chain.invoke("What is macronutrients?"))

Both `max_new_tokens` (=1024) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Human: 
<|system|>>
You are a helpful AI Assistant that follows instructions extremely well.
Use the following context to answer user question.

Think step by step before answering the question. You will get a $100 tip if you provide correct answer.

CONTEXT: [Document(metadata={'source': '/content/pdf-data/human-nutrition-text.pdf'}, page_content='use filters to purify their environment from electrical emissions.\n\n7. Mathieu J. (2005). What is orthorexia? Journal of the\n\nAmerican Dietetic Association, 105(10), 1510-1512.\n\nBratman, S. Health Food Junkie. Yoga Journal. 1997,\n\nSeptember/October, 42-50. Available at\n\nhttps://www.orthorexia.com/original-orthorexia-\n\nessay/.\n\n8. Donini LM, Marsili D, Graziani MP, Imbriale M, Cannella\n\nC. (2004). Orthorexia nervosa: a preliminary study with a\n\nproposal for diagnosis and an attempt to measure the\n\ndimension of the phenomenon. Eating and Weight\n\nDisorders, 9(2), 151‐157.\n\n9. Orthorexia. (2017, February 26). National Eat

In [36]:
print(normal_chain.invoke("What is macronutrients?"))

Both `max_new_tokens` (=1024) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Human: 
<|system|>>
You are a helpful AI Assistant that follows instructions extremely well.
Use the following context to answer user question.

Think step by step before answering the question. You will get a $100 tip if you provide correct answer.

CONTEXT: [Document(metadata={'source': '/content/pdf-data/human-nutrition-text.pdf'}, page_content='lipids, proteins, water, vitamins, and minerals. Foods also contain\n\nnon-nutrients that may be harmful (such as natural toxins common\n\nin plant foods and additives like some dyes and preservatives) or\n\nbeneficial (such as antioxidants).\n\n4 | Introduction\n\nMacronutrients\n\nNutrients\n\nthat are needed\n\nin\n\nlarge amounts are called\n\nmacronutrients. There are three classes of macronutrients:\n\ncarbohydrates, lipids, and proteins. These can be metabolically\n\nprocessed into cellular energy. The energy from macronutrients\n\ncomes from their chemical bonds. This chemical energy is\n\nconverted into cellular energy that is then 

<strong>Hybrid Search</strong>

Macronutrients are nutrients that are needed by the body in large quantities, typically measured in grams or ounces. There are three major classes of macronutrients: carbohydrates, lipids (fats), and proteins. Macronutrients provide the body with energy (calories) that can be utilized to perform various bodily functions. The energy derived from these nutrients comes from their chemical bonds, and a unit of measurement for food energy is the calorie (kcal). While water is also a macronutrient due to its large quantity requirements, it does not provide calories.




<strong> Embedding-Based Search</strong>

Macronutrients are nutrients that are needed by the body in large amounts, typically measured in grams or ounces. They provide the body with energy and include three classes: carbohydrates, lipids, and proteins. Carbohydrates are found in foods like grains, fruits, and vegetables and are used to produce energy. Lipids, also known as fats, are found in foods like oils, nuts, and meats and are used for energy as well as for building cell membranes. Proteins are found in foods like meat, beans, and eggs and are used for building and repairing tissues in the body. Water is also considered a macronutrient because it must be consumed in large quantities to support bodily functions. Unlike macronutrients, water does not provide energy but is essential for life's basic processes.

## Question with general context without abbreviations or keywords

In [37]:
# page 60
print(normal_chain.invoke("What are units of measure used in nutrition?"))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Both `max_new_tokens` (=1024) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Human: 
<|system|>>
You are a helpful AI Assistant that follows instructions extremely well.
Use the following context to answer user question.

Think step by step before answering the question. You will get a $100 tip if you provide correct answer.

CONTEXT: [Document(metadata={'source': '/content/pdf-data/human-nutrition-text.pdf'}, page_content='Learning Activities\n\nTechnology Note: The second edition of the Human\n\nNutrition Open Educational Resource (OER) textbook\n\nfeatures interactive learning activities. These activities are\n\navailable in the web-based textbook and not available in the\n\n16 | Food Quality\n\ndownloadable versions (EPUB, Digital PDF, Print_PDF, or\n\nOpen Document).\n\nLearning activities may be used across various mobile\n\ndevices, however, for the best user experience it is strongly\n\nrecommended that users complete these activities using a\n\ndesktop or laptop computer and in Google Chrome.\n\nAn interactive or media element has been\n\nexcluded from

In [38]:
print(hybrid_chain.invoke("What are units of measure used in nutrition?"))

Both `max_new_tokens` (=1024) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Human: 
<|system|>>
You are a helpful AI Assistant that follows instructions extremely well.
Use the following context to answer user question.

Think step by step before answering the question. You will get a $100 tip if you provide correct answer.

CONTEXT: [Document(metadata={'source': '/content/pdf-data/human-nutrition-text.pdf'}, page_content='released to provide kinetic energy, it ultimately becomes thermal\n\nenergy or heat. You can notice this when you exercise and your\n\nbody heats up.\n\nThe Calorie Is a Unit of Energy\n\nThe amount of energy in nutrients or the amount of energy\n\nexpended by the body can be quantified with a variety of units\n\nused to measure energy. In the US, the kilocalorie (kcal) is most\n\ncommonly used and is often just referred to as a calorie. Strictly\n\nspeaking, a kcal is 1000 calories. In nutrition, the term calories\n\nIntroduction | 455\n\nalmost always refers to kcals. Sometimes the kcal is indicated by\n\ncapitalizing calories as “Calories

<strong>Embeddings-based search's output</strong>
There are two systems commonly used in nutrition for measuring ingredients and nutrients: the Metric System and the US Customary System (also known as Imperial or British units). The following prefixes are commonly used in the Metric System:

- Micro (μ): one millionth
- Milli (m): one thousandth
- Centi (c): one hundredth
- Deci (d): one tenth
- Kilo (k): one thousand times

Examples of conversions between the two systems include:

- Microgram (μg): Ounce (oz)
  - 1 oz = 28.35 g
- Milligram (mg): Pound (lb)
  - 1 lb = 16 oz
- Gram (g):
- Kilogram (kg)

Some commonly used units of measurement in nutrition include:

- Calorie (kcal or Calorie)
- Milligrams (mg)
- Micrograms (μg)
- International Units (IU)
- Millimoles (mmol)
- Liters (L)
- Milliliters (ml)
- Grams (g)
- Kilograms (kg)
- Pounds (lb)

The updated Daily Values (DV) for various nutrients have also recently changed, with decreases for some vitamins and minerals, and an introduction of the term "Added sugars" to the label. These changes will be phased in over the next few years, with compliance deadlines for different-sized manufacturing companies.

<strong>Hybrid Search Answer</strong>
In nutrition, there are two systems of commonly used measurements: metric and US customary. Some commonly used prefixes for the Metric System include:

- Micro- (μ) for 1/1,000,000th (one millionth)
- Milli- (m) for 1/1,000th (one thousandth)
- Centi- (c) for 1/100th (one hundredth)
- Deci- (d) for 1/10th (one tenth)
- Kilo- (k) for 1,000 times

The metric system uses units such as micrograms (μg), milligrams (mg), grams (g), kilograms (kg), milliliters (mL), and liters (L) for mass, volume, and capacity, respectively.

In the US customary system, units include:

- Pound (lb) for mass
- Ounce (oz) for mass and volume
- Quart (qt) for volume
- Gallon (gal) for volume

Some commonly used conversions between the two systems are:

- 1 ounce (oz) = 28.35 grams (g)
- 1 pound (lb) = 16 ounces (oz)

Additionally, a unit of measurement of food energy is the calorie, with one kilocalorie (kcal) equivalent to 1,000 calories (denoted with a small “c”). On nutrition food labels, the amount given for “calories” is actually equivalent to each calorie multiplied by one thousand. The "Calorie" (with a capital “C”) on nutrition food labels is synonymous with a kilocalorie (kcal).

## Question contains a specific names of food and name filters

In [39]:
# page 272 nutrition book
print(normal_chain.invoke("What other carbohydrate-rich foods did the ancient Hawaiians consume beside uala?"))

Both `max_new_tokens` (=1024) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Human: 
<|system|>>
You are a helpful AI Assistant that follows instructions extremely well.
Use the following context to answer user question.

Think step by step before answering the question. You will get a $100 tip if you provide correct answer.

CONTEXT: [Document(metadata={'source': '/content/pdf-data/human-nutrition-text.pdf'}, page_content='Describe the process of carbohydrate digestion and\n\nabsorption\n\n\n\nDescribe the functions of carbohydrates in the\n\nbody\n\n\n\nDescribe the body’s carbohydrate needs and how\n\npersonal choices can lead to health benefits or\n\nconsequences\n\nThroughout history, carbohydrates have and continue to be a major\n\nsource of people’s diets worldwide. In ancient Hawai‘i the Hawaiians\n\nobtained the majority of their calories from carbohydrate rich plants\n\nlike the ‘uala (sweet potato), ulu (breadfruit) and kalo (taro). For\n\nexample, mashed kalo or poi was a staple to meals for Hawaiians.\n\nResearch suggests that almost 78 percent of 

In [40]:
print(hybrid_chain.invoke("What other carbohydrate-rich foods did the ancient Hawaiians consume beside uala?"))

Both `max_new_tokens` (=1024) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Human: 
<|system|>>
You are a helpful AI Assistant that follows instructions extremely well.
Use the following context to answer user question.

Think step by step before answering the question. You will get a $100 tip if you provide correct answer.

CONTEXT: [Document(metadata={'source': '/content/pdf-data/human-nutrition-text.pdf'}, page_content='Describe the process of carbohydrate digestion and\n\nabsorption\n\n\n\nDescribe the functions of carbohydrates in the\n\nbody\n\n\n\nDescribe the body’s carbohydrate needs and how\n\npersonal choices can lead to health benefits or\n\nconsequences\n\nThroughout history, carbohydrates have and continue to be a major\n\nsource of people’s diets worldwide. In ancient Hawai‘i the Hawaiians\n\nobtained the majority of their calories from carbohydrate rich plants\n\nlike the ‘uala (sweet potato), ulu (breadfruit) and kalo (taro). For\n\nexample, mashed kalo or poi was a staple to meals for Hawaiians.\n\nResearch suggests that almost 78 percent of 

<strong>Embeddings-based search</strong>

In addition to uala (sweet potato), ancient Hawaiians also obtained carbohydrates from foods such as ulu (breadfruit) and kalo (taro). According to the research cited in the context, almost 78% of their diet was made up of these fiber-rich carbohydrate foods. Other carbohydrate-rich foods that may have been consumed by ancient Hawaiians include bananas, coconut, yams, and various other root vegetables.



<strong>Hybrid Search</strong>

In addition to uala (sweet potatoes), ancient Hawaiians also consumed other carbohydrate-rich foods such as kalo (taro), 'ulu (breadfruit), 'ipu (bananas), mamala (pounded yams), and awa (cassava). These crops were cultivated through traditional farming practices and played a significant role in the traditional Hawaiian diet and culture.

## Question contains a specific locations

In [41]:
# page 98 nutrition book
print(normal_chain.invoke("In the late 19th century, where were the tribe that fascinated Elie Metchnikoff due to their longevity located?"))

Both `max_new_tokens` (=1024) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Human: 
<|system|>>
You are a helpful AI Assistant that follows instructions extremely well.
Use the following context to answer user question.

Think step by step before answering the question. You will get a $100 tip if you provide correct answer.

CONTEXT: [Document(metadata={'source': '/content/pdf-data/human-nutrition-text.pdf'}, page_content='Learning Objectives\n\nBy the end of this chapter, you will be able to:\n\nIntroduction | 55\n\n\n\nExplain the anatomy and physiology of the\n\ndigestive system and other supporting organ systems\n\n\n\nDescribe the relationship between diet and each of\n\nthe organ systems\n\n\n\nDescribe the process of calculating Body Mass\n\nIndex (BMI)\n\nThe Native Hawaiians believed there was a strong connection\n\nbetween health and food. Around the world, other cultures had\n\nsimilar views of food and its relationship with health. A famous\n\nquote by the Greek physician Hippocrates over two thousand years\n\nago, “Let food be thy medicine and med

In [42]:
# page 98 nutrition book
print(hybrid_chain.invoke("In the late 19th century, where were the tribe that fascinated Elie Metchnikoff due to their longevity located?"))

Both `max_new_tokens` (=1024) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Human: 
<|system|>>
You are a helpful AI Assistant that follows instructions extremely well.
Use the following context to answer user question.

Think step by step before answering the question. You will get a $100 tip if you provide correct answer.

CONTEXT: [Document(metadata={'source': '/content/pdf-data/human-nutrition-text.pdf'}, page_content='Learning Objectives\n\nBy the end of this chapter, you will be able to:\n\nIntroduction | 55\n\n\n\nExplain the anatomy and physiology of the\n\ndigestive system and other supporting organ systems\n\n\n\nDescribe the relationship between diet and each of\n\nthe organ systems\n\n\n\nDescribe the process of calculating Body Mass\n\nIndex (BMI)\n\nThe Native Hawaiians believed there was a strong connection\n\nbetween health and food. Around the world, other cultures had\n\nsimilar views of food and its relationship with health. A famous\n\nquote by the Greek physician Hippocrates over two thousand years\n\nago, “Let food be thy medicine and med

<strong>Embeddings-based search</strong>

The text mentions that around the latter nineteenth century, Elie Metchnikoff was intrigued by the healthy life spans and resistance to life-threatening diseases in people who lived in the tribes of the northern Caucasus Mountains. Therefore, the tribe that fascinated Elie Metchnikoff due to their longevity was located in the northern Caucasus Mountains.

<strong>Hybrid Search</strong>

The passage mentions that Elie Metchnikoff became intrigued by the healthy life spans of people in the tribes of the northern Caucasus Mountains. This region is located at the border of Russia, Georgia, and Turkey. Specific tribes that Metchnikoff may have been referring to include the Circassians, Chetschenes, and Kabardians.